In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
from joblib import (Parallel,
                    delayed)

from src.Tools import portfolio_metrics
from qiskit_src.ansatz import CP_QAOA, Grid
from qiskit_src.tools import get_qubo, min_cost_partition, normalized_cost

plt.rc("font", family=["Helvetica", "Arial"])
plt.rc("text", usetex=True)
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}\usepackage{amssymb}\usepackage{physics}'
plt.rc("xtick", top=True, direction="in")
plt.rc("ytick", right=True, direction="in")

In [4]:
N_CPU_CORES = os.cpu_count()
N_CPU_CORES

16

In [ ]:
def simulate(datapoint: tuple):    
    _seed_,_N_,_k_,_alpha_,_layers_,_max_iter_,_w_z_phase_,_w_next_nearest_neighbors_,_w_evenly_distributed_, _w_grid_, _method_ = datapoint
    expected_returns, covariances = portfolio_metrics(n=_N_, seed=_seed_)

    constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=_N_,
                                                                k=_k_,
                                                                mu=expected_returns,
                                                                sigma=covariances,
                                                                alpha=_alpha_)
    
    max_cost, min_cost, min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
    Q, offset = get_qubo(mu=expected_returns,
                         sigma=covariances,
                         alpha=_alpha_,
                         lmbda=lmbda,
                         k=_k_)
    if _w_grid_:
        my_grid = Grid(N_qubits=_N_)
    ansatz = CP_QAOA(N_qubits=_N_,
                     cardinality=_k_,
                     layers=_layers_,
                     grid=my_grid if _w_grid_ else None,
                     QUBO_matrix=Q,
                     QUBO_offset=offset,
                     with_next_nearest_neighbors=_w_next_nearest_neighbors_,
                     with_evenly_distributed_start_x=_w_evenly_distributed_,
                     with_z_phase=_w_z_phase_)
    if _w_grid_:
        N_xx_yy_angles = _layers_ * len(my_grid.get_NN_indices())
    else:
        N_xx_yy_angles = _layers_ * (_N_ - 1) 
    if _w_next_nearest_neighbors_:
        N_xx_yy_angles += _layers_ * (_N_ - 2)
    if _w_z_phase_:
        N_xx_yy_angles += _N_ * _layers_
    theta_i = np.random.normal(loc=0, scale=0.1, size=N_xx_yy_angles)
    iteration_dicts = []

    def callback_function(x):
        iteration_dicts.append(ansatz.get_state_probabilities(angles=x, flip_states=False))
    res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                               method=_method_,
                               options={'disp': False, 'maxiter': _max_iter_},
                               callback=callback_function)
    return [normalized_cost(result=it,
                            QUBO_matrix=Q,
                            QUBO_offset=offset,
                            max_cost=max_cost, 
                            min_cost=min_cost) for it in iteration_dicts]

In [ ]:
# General settings:
N=9
k=4
alpha=0.001
layers=2
max_iter=150
w_z_phase = False
w_next_nearest_neighbors = True
w_evenly_distributed = True
method_1='COBYLA'
method_2='Nelder-Mead'
method_3='SPSA'

# Number of runs:
N_SEEDS = (N_CPU_CORES-1)*10
datapoints_1, datapoints_2, datapoints_3 = [], [], []
for seed in range(N_SEEDS):
    datapoints_1.append((seed,N,k,alpha,layers,max_iter,w_z_phase,w_next_nearest_neighbors,w_evenly_distributed,method_1))
    datapoints_2.append((seed,N,k,alpha,layers,max_iter,w_z_phase,w_next_nearest_neighbors,w_evenly_distributed,method_2))
    datapoints_3.append((seed,N,k,alpha,layers,max_iter,w_z_phase,w_next_nearest_neighbors,w_evenly_distributed,method_3))

N_jobs = N_CPU_CORES-1
YS_1 = Parallel(n_jobs=N_jobs, verbose=51, backend='loky')(delayed(simulate)(datapoint) for datapoint in datapoints_1)
YS_2 = Parallel(n_jobs=N_jobs, verbose=51, backend='loky')(delayed(simulate)(datapoint) for datapoint in datapoints_2)
YS_3 = Parallel(n_jobs=N_jobs, verbose=51, backend='loky')(delayed(simulate)(datapoint) for datapoint in datapoints_3)